# Cache Stampede Benchmark Results

Analysis of cache stampede protection across different cache implementations.

In [11]:
import pandas as pd
import altair as alt

# Enable theme
alt.theme.enable("latimes")

# Load ConcurrentDictionary benchmark data
df_cd = pd.read_csv("./Benchmark.ConcurrentDictionaryBenchmark-report.csv")

## ConcurrentDictionary Benchmark

ConcurrentDictionary without stampede protection - each concurrent request executes the expensive operation.

In [12]:
def clean_benchmark_data(df):
    """Clean and prepare benchmark data for visualization"""
    # Clean Median column (remove 'ms' and convert to float)
    df["Median"] = df["Median"].str.replace(" ms", "", case=False, regex=False).str.replace(",", "", case=False, regex=False)
    df = df.astype({"Median": "float64"})
    
    # Clean Allocated column (remove 'B' and convert to float, then to KB)
    df["Allocated"] = df["Allocated"].str.replace(" B", "", case=False, regex=False).str.replace(",", "", case=False, regex=False)
    df = df.astype({"Allocated": "float64"})
    df["Allocated_KB"] = df["Allocated"] / 1024  # Convert to KB
    
    return df

# Clean the data
df_cd_clean = clean_benchmark_data(df_cd.copy())

# Display sample
df_cd_clean[["Operation", "ConcurrentRequests", "Median", "Allocated_KB", "Op Count"]].head(10)

,Operation,ConcurrentRequests,Median,Allocated_KB,Op Count
0,IOBound,1,200.8,0.468750,1.0
1,IOBound,2,200.7,0.804688,2.0
2,IOBound,3,200.7,1.070312,3.0
3,IOBound,4,200.8,1.335938,4.0
4,IOBound,5,200.7,1.601562,5.0
5,IOBound,6,200.7,1.867188,6.0
6,IOBound,7,200.7,2.132812,7.0
7,IOBound,8,200.7,2.398438,8.0
8,IOBound,9,200.8,2.664062,9.0
9,IOBound,10,200.8,2.929688,10.0


## Create 

In [13]:
def create_single_replica_chart(
    df, title, colorColumn="Operation", colorTitle="Operation"
):
    """Create a chart with 3 subplots for a specific operation type"""

    # Base chart configuration
    base = alt.Chart(df).encode(
        x=alt.X("ConcurrentRequests:Q", title="Concurrent Requests"),
        color=alt.Color(f"{colorColumn}:N", title=colorTitle),
    )

    width = 245
    height = 316

    # Chart 1: Median execution time
    median_chart = (
        base.mark_line(point=alt.OverlayMarkDef(shape="cross"))
        .encode(
            y=alt.Y("Median:Q", title="Median execution time (ms)"),
        )
        .properties(width=width, height=height)
    )

    # Chart 2: Allocated memory in KB
    allocated_chart = (
        base.mark_line(point=alt.OverlayMarkDef(shape="circle"))
        .encode(
            y=alt.Y("Allocated_KB:Q", title="Allocated Memory (KB)"),
        )
        .properties(width=width, height=height)
    )

    # Chart 3: Op Count
    opcount_chart = (
        base.mark_line(point=alt.OverlayMarkDef(shape="square"))
        .encode(
            y=alt.Y("Op Count:Q", title="Operations executed"),
        )
        .properties(width=width, height=height)
    )

    # Combine horizontally
    combined = (median_chart | allocated_chart | opcount_chart).properties(
        title={"text": title, "fontWeight": "bold"}
    )

    return combined

In [14]:
# Create charts for CPU-bound and IO-bound
chart_cd = create_single_replica_chart(df_cd_clean, "ConcurrentDictionary")

chart_cd.save("results_cd.png", scale_factor=1)

chart_cd

alt.HConcatChart(...)

## MemoryCache Benchmark

IMemoryCache without stampede protection - each concurrent request executes the expensive operation.

In [15]:
# Load MemoryCache benchmark data
df_mc = pd.read_csv("./Benchmark.MemoryCacheBenchmark-report.csv")
df_mc_clean = clean_benchmark_data(df_mc.copy())

# Create charts for MemoryCache
chart_mc = create_single_replica_chart(df_mc_clean, "MemoryCache")

chart_mc.save("results_mc.png", scale_factor=1)

chart_mc

alt.HConcatChart(...)

## HybridCache Benchmark

HybridCache with L1 (in-memory) + optional L2 (distributed) cache. Provides stampede protection - only one operation executes per replica.

In [16]:
# Load HybridCache benchmark data
df_hc = pd.read_csv("./Benchmark.HybridCacheBenchmark-report.csv")
df_hc_clean = clean_benchmark_data(df_hc.copy())

In [19]:
# Filter for L1Only mode
df_hc_l1_r1 = df_hc_clean[
    (df_hc_clean["Mode"] == "L1Only") & (df_hc_clean["ReplicasCount"] == 1)
    # & (df_hc_clean["Operation"] == "CPUBound")
].copy()

# Create charts for L1Only CPU-bound and IO-bound
chart_hc_l1_cpu = create_single_replica_chart(
    df_hc_l1_r1,
    "HybridCache L1 Only",
    # "ReplicasCount", "Replicas Count"
)

chart_hc_l1_cpu.save("results_hc_l1.png", scale_factor=1)

chart_hc_l1_cpu

alt.HConcatChart(...)

In [21]:
df_hc_l1_r1 = df_hc_clean[
    (df_hc_clean["Mode"] == "L1L2") & (df_hc_clean["ReplicasCount"] == 2)
].copy()

chart_hc_l1l2_cpu = create_single_replica_chart(
    df_hc_l1_r1, "HybridCache L1 + L2" #, "ReplicasCount", "Replicas Count"
)

chart_hc_l1l2_cpu.save("results_hc_l1l2.png", scale_factor=1)

chart_hc_l1l2_cpu

alt.HConcatChart(...)